In [1]:
from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession.builder.appName("Regression")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/23 14:21:28 WARN Utils: Your hostname, DESKTOP-EMCMOU9 resolves to a loopback address: 127.0.1.1; using 172.20.189.237 instead (on interface eth0)
22/09/23 14:21:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/23 14:21:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample

In [2]:
# read in datasets
merchant_fraud = spark.read.parquet('../data/curated/merchant_fraud.parquet')
consumer_fraud = spark.read.parquet('../data/curated/consumer_fraud.parquet')

In [3]:
merchant_fraud.limit(10)

merchant_abn,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
38700038932,2021-08-20,14,1524.9292206627556,false,0.01
89502033586,2021-08-20,40,65.44754598876243,false,0.01
70610974780,2021-08-20,27,43.48907978704201,false,0.01
29550468444,2021-08-20,4,157.0,false,0.01
92075595936,2021-08-20,12,95.50018824235849,false,0.01
62422038748,2021-08-20,1,2442.1698743679544,false,0.01
32894483068,2021-08-20,1,228.6561829721828,false,0.01
55958301764,2021-08-20,6,281.354343839723,false,0.01
12543580354,2021-08-20,12,262.53675367314503,false,0.01
26008308191,2021-08-20,6,241.04606090655918,false,0.01


In [4]:
merchant_fraud.count()

4033545

In [5]:
merchant_fraud.where(F.col('fraud_probability') != 0.01).count()

342

About 0.008% of the merchant daily transactions is fraudulent.

In [6]:
consumer_fraud.limit(10)

user_id,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
18487,2021-08-20,2,73.0925761125627,false,0.01
18552,2021-08-20,2,174.42985218811984,false,0.01
215,2021-08-20,2,48.880342827706,false,0.01
18770,2021-08-20,1,273.2147324712209,false,0.01
18837,2021-08-20,1,23.800263170555642,false,0.01
18849,2021-08-20,3,56.899667578706065,false,0.01
435,2021-08-20,2,21.310166954483677,false,0.01
788,2021-08-20,2,38.272827709859286,false,0.01
19550,2021-08-20,2,17.85711450212551,false,0.01
1117,2021-08-20,3,57.4104700047829,false,0.01


In [7]:
consumer_fraud.count()

17954112

In [8]:
consumer_fraud.where(F.col('fraud_probability') != 0.01).count()

69728

About 0.4% of the consumer daily transactions is fraudulent.

Since the proportion of non-fraud to fraud data in both merchant and consumer fraud datasets is highly imbalanced, resampling is performed on both datasets before fitting a model.

For both training datasets, the proportion used is 550 samples of non-fraud data and 450 samples of fraud data. The proportion of non-fraud data is slightly higher since it represents the majority of the data.

In [32]:
def resampling(majority, minority):
    """
    Performs undersampling on the majority class (spark dataframe)
    and oversampling on the minority class (pandas dataframe) from an imbalanced dataset. 
    Outputs resampled dataset as a pandas dataframe.
    """
    
    # define number of samples needed from each class
    N_MAJOR = 550
    N_MINOR = 450
    FRAC_MAJOR = N_MAJOR / majority.count()
    
    # repeated sampling from the minority class
    n_more_samples = N_MINOR - minority.shape[0] # needs to be less than no. of samples in minority
    if n_more_samples > 0:
        sampled_minority = pd.concat([minority, resample(minority, replace=True, n_samples=n_more_samples, random_state=0)], axis=0)
    else:
        sampled_minority = minority
    
    # undersampling from the majority class
    sampled_majority = majority.sample(FRAC_MAJOR, seed=0).toPandas()
    
    # join the sampled datasets
    resampled = pd.concat([sampled_majority, sampled_minority], axis=0)
    
    return resampled

## Merchant fraud data

### Resampling data

In [10]:
# split merchant fraud data by class
merchant_majority = merchant_fraud.where(F.col('fraud_probability') == 0.01)
merchant_minority = merchant_fraud.where(F.col('fraud_probability') != 0.01).toPandas()

In [42]:
# obtain resampled dataset for modelling
merchant_resampled = resampling(merchant_majority, merchant_minority)

In [43]:
merchant_resampled.head()

,merchant_abn,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
0,15269431008,2021-06-25,16,94.062315,False,0.01
1,26603390734,2021-08-13,2,654.763018,False,0.01
2,66917621463,2021-05-16,4,331.084177,False,0.01
3,92681936901,2021-07-21,6,259.949417,False,0.01
4,89022505272,2021-07-20,5,131.199399,False,0.01


### Feature engineering 

In [44]:
# parse order datetime column to pandas' datetime
merchant_resampled['order_datetime'] = pd.to_datetime(merchant_resampled['order_datetime'])

In [45]:
# create order day of week column - Monday = 0, Sunday = 6
merchant_resampled['order_dayofweek'] = merchant_resampled['order_datetime'].dt.day_of_week

In [46]:
# one-hot encoding on order day of week column
merchant_resampled = pd.get_dummies(data=merchant_resampled, prefix='dow', columns=['order_dayofweek'])

merchant_resampled.head()

,merchant_abn,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,15269431008,2021-06-25,16,94.062315,False,0.01,0,0,0,0,1,0,0
1,26603390734,2021-08-13,2,654.763018,False,0.01,0,0,0,0,1,0,0
2,66917621463,2021-05-16,4,331.084177,False,0.01,0,0,0,0,0,0,1
3,92681936901,2021-07-21,6,259.949417,False,0.01,0,0,1,0,0,0,0
4,89022505272,2021-07-20,5,131.199399,False,0.01,0,1,0,0,0,0,0


In [47]:
# split dataset into predictors and target variables
X = pd.concat([merchant_resampled.iloc[:,2:4], merchant_resampled.iloc[:,6:]], axis=1)
y = merchant_resampled.iloc[:,4]

In [48]:
X.head()

,transaction_count,avg_transaction_amt,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,16,94.062315,0,0,0,0,1,0,0
1,2,654.763018,0,0,0,0,1,0,0
2,4,331.084177,0,0,0,0,0,0,1
3,6,259.949417,0,0,1,0,0,0,0
4,5,131.199399,0,1,0,0,0,0,0


In [49]:
# convert target variable from boolean to integers
y = y.replace({True: 1, False: 0})

### Model fitting

In [52]:
# cross validation
lgr_merchant = LogisticRegression(max_iter=250)
cv_scores = cross_val_score(lgr_merchant, X, y, cv=3)
print(cv_scores)
print('Average cross-validation score: ', np.mean(cv_scores))

[0.9        0.93617021 0.94528875]
Average cross-validation score:  0.9271529888551164


/home/seenyanl/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Consumer fraud data

Apply the same pipeline to consumer fraud dataset.

### Resampling data

In [22]:
# split consumer fraud data by class
consumer_majority = consumer_fraud.where(F.col('fraud_probability') == 0.01)
consumer_minority = consumer_fraud.where(F.col('fraud_probability') != 0.01).toPandas()

In [33]:
# obtain resampled dataset for modelling
consumer_resampled = resampling(consumer_majority, consumer_minority)

22/09/23 14:48:16 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 172.20.189.237:33067 in 10000 milliseconds


In [34]:
consumer_resampled.head()

,user_id,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
0,15788,2021-08-15,1,201.365642,False,0.01
1,21853,2021-08-27,1,16.136864,False,0.01
2,11998,2021-08-18,1,399.775259,False,0.01
3,12029,2021-06-18,2,28.930171,False,0.01
4,960,2021-08-03,2,47.653247,False,0.01


### Feature engineering

In [35]:
# parse order datetime column to pandas' datetime
consumer_resampled['order_datetime'] = pd.to_datetime(consumer_resampled['order_datetime'])

In [36]:
# create order day of week column - Monday = 0, Sunday = 6
consumer_resampled['order_dayofweek'] = consumer_resampled['order_datetime'].dt.day_of_week

In [37]:
# one-hot encoding on order day of week column
consumer_resampled = pd.get_dummies(data=consumer_resampled, prefix='dow', columns=['order_dayofweek'])

consumer_resampled.head()

,user_id,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,15788,2021-08-15,1,201.365642,False,0.01,0,0,0,0,0,0,1
1,21853,2021-08-27,1,16.136864,False,0.01,0,0,0,0,1,0,0
2,11998,2021-08-18,1,399.775259,False,0.01,0,0,1,0,0,0,0
3,12029,2021-06-18,2,28.930171,False,0.01,0,0,0,0,1,0,0
4,960,2021-08-03,2,47.653247,False,0.01,0,1,0,0,0,0,0


In [38]:
# split dataset into predictors and target variables
X = pd.concat([consumer_resampled.iloc[:,2:4], consumer_resampled.iloc[:,6:]], axis=1)
y = consumer_resampled.iloc[:,4]

In [39]:
X.head()

,transaction_count,avg_transaction_amt,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,1,201.365642,0,0,0,0,0,0,1
1,1,16.136864,0,0,0,0,1,0,0
2,1,399.775259,0,0,1,0,0,0,0
3,2,28.930171,0,0,0,0,1,0,0
4,2,47.653247,0,1,0,0,0,0,0


In [40]:
# convert target variable from boolean to integers
y = y.replace({True: 1, False: 0})

In [41]:
lgr_consumer = LogisticRegression(max_iter=250)
cv_scores = cross_val_score(lgr_consumer, X, y, cv=3)
print(cv_scores)
print('Average cross-validation score: ', np.mean(cv_scores))

[0.99953044 0.999317   0.9996158 ]
Average cross-validation score:  0.9994877431884102


## Generating predictions for full datasets

In [ ]:
df_merchant = pd.read_parquet('../data/curated/merchant_fraud.parquet')
df_consumer = pd.read_parquet('../data/curated/consumer_fraud.parquet')

In [ ]:
# df_merchant = merchant_fraud.toPandas()
# df_consumer = consumer_fraud.toPandas()

Predictions for merchant.

In [ ]:
# order day of week feature engineering
df_merchant['order_datetime'] = pd.to_datetime(df_merchant['order_datetime'])
df_merchant['order_dayofweek'] = df_merchant['order_datetime'].dt.day_of_week
df_merchant = pd.get_dummies(data=df_merchant, prefix='dow', columns=['order_dayofweek'])

df_merchant.head()

In [ ]:
# get predictor and target variables
X_merchant = pd.concat([df_merchant.iloc[:,2:4], df_merchant.iloc[:,6:]], axis=1)
y_merchant = df_merchant.iloc[:,4]
y_merchant = y_merchant.replace({True: 1, False: 0})

In [ ]:
# get predictions
y_merchant_pred = lgr_merchant.predict(X_merchant)
print(lgr_merchant.score(y_merchant, y_merchant_pred))

Predictions for consumer.

In [ ]:
# order day of week feature engineering
df_consumer['order_datetime'] = pd.to_datetime(df_consumer['order_datetime'])
df_consumer['order_dayofweek'] = df_consumer['order_datetime'].dt.day_of_week
df_consumer = pd.get_dummies(data=df_consumer, prefix='dow', columns=['order_dayofweek'])

df_consumer.head()

In [ ]:
# get predictor and target variables
X_consumer = pd.concat([df_consumer.iloc[:,2:4], df_consumer.iloc[:,6:]], axis=1)
y_consumer = df_consumer.iloc[:,4]
y_consumer = y_consumer.replace({True: 1, False: 0})

In [ ]:
# get predictions
y_consumer_pred = lgr_consumer.predict(X_consumer)
print(lgr_consumer.score(y_consumer, y_consumer_pred))